In [1]:
import tensorflow as tf
import numpy as np

In [2]:
learning_rate = 0.01

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')

layer계층 별로 정리를 해준 후 그래프를 보여주는 방법

tf.name_scope("layer이름")을 이용하면 각각의 layer의 구조를 보여줌

In [4]:
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2, 2]), name='weight1')
    b1 = tf.Variable(tf.random_normal([2]), name='bias1')
    layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)
    
    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)

with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2, 1]), name='weight2')
    b2 = tf.Variable(tf.random_normal([1]), name='bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, W2) + b2)
    
    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)

In [5]:
# cost/loss function
with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)
    
with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [7]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs_r0_01")
    writer.add_graph(sess.graph) # Show the graph
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, global_step=step)
        
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run([W1, W2]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)
            

0 0.918302 [array([[ 0.37596378, -0.28133816],
       [ 0.21789296,  1.00692511]], dtype=float32), array([[-0.85134268],
       [ 0.75133133]], dtype=float32)]
100 0.684973 [array([[ 0.01272922,  0.55740362],
       [-0.13428959,  1.55899763]], dtype=float32), array([[-0.41196209],
       [ 1.22839904]], dtype=float32)]
200 0.66739 [array([[  1.95423678e-01,   1.28797233e+00],
       [  1.15671661e-03,   2.28215504e+00]], dtype=float32), array([[-0.44891098],
       [ 1.31291294]], dtype=float32)]
300 0.629421 [array([[ 0.44469383,  2.18463683],
       [ 0.06746759,  3.30843401]], dtype=float32), array([[-0.53867114],
       [ 1.61063886]], dtype=float32)]
400 0.578762 [array([[ 1.04106212,  3.22777677],
       [-0.1340846 ,  4.4103446 ]], dtype=float32), array([[-0.74572498],
       [ 2.09720349]], dtype=float32)]
500 0.513484 [array([[ 2.7568357 ,  4.16774988],
       [-1.43769407,  5.26654959]], dtype=float32), array([[-1.16365504],
       [ 2.62199402]], dtype=float32)]
600 0.44747

5000 0.347151 [array([[ 13.08567524,  11.50803757],
       [ -7.62103176,   7.99518394]], dtype=float32), array([[-7.27522039],
       [ 7.96465683]], dtype=float32)]
5100 0.347118 [array([[ 13.16361046,  11.58316135],
       [ -7.64907646,   8.0138855 ]], dtype=float32), array([[-7.33434057],
       [ 8.01977253]], dtype=float32)]
5200 0.347087 [array([[ 13.24080658,  11.65767097],
       [ -7.67667341,   8.03238678]], dtype=float32), array([[-7.39301634],
       [ 8.07452393]], dtype=float32)]
5300 0.347058 [array([[ 13.31730938,  11.73160744],
       [ -7.70383787,   8.05069542]], dtype=float32), array([[-7.4512682 ],
       [ 8.12892723]], dtype=float32)]
5400 0.347031 [array([[ 13.39315128,  11.80499554],
       [ -7.73058176,   8.06882   ]], dtype=float32), array([[-7.50911999],
       [ 8.18300724]], dtype=float32)]
5500 0.347006 [array([[ 13.46836758,  11.87787056],
       [ -7.75693941,   8.08677006]], dtype=float32), array([[-7.56659698],
       [ 8.23678207]], dtype=float32)

9900 0.346615 [array([[ 16.43538284,  14.79934788],
       [ -8.67770386,   8.76891041]], dtype=float32), array([[ -9.90676117],
       [ 10.45870781]], dtype=float32)]
10000 0.346613 [array([[ 16.49837303,  14.86212349],
       [ -8.69524574,   8.78277493]], dtype=float32), array([[ -9.95788765],
       [ 10.5078373 ]], dtype=float32)]

Hypothesis:  [[  5.45802723e-05]
 [  9.99937654e-01]
 [  4.99992192e-01]
 [  5.00012934e-01]] 
Correct:  [[ 0.]
 [ 1.]
 [ 0.]
 [ 1.]] 
Accuracy:  0.5
